In [2]:
#@title Insatlling Pyorch

!pip install torch
!pip install torchvision

In [3]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [4]:
#@title Define Hyperparameters

input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 20 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step

In [5]:
#@title Downloading MNIST data

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 112302137.54it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 101114936.41it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 32859586.24it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15071620.86it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

In [7]:
#@title Define model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [8]:
#@title Build the model

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [9]:
#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [12]:
#@title Training the model

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'%(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))
    # if (i+1) % 100 == 0:
      # print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                #  %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data[0]))

Epoch [1/20], Step [100/600], Loss: 0.2051
Epoch [1/20], Step [200/600], Loss: 0.1121
Epoch [1/20], Step [300/600], Loss: 0.1692
Epoch [1/20], Step [400/600], Loss: 0.2673
Epoch [1/20], Step [500/600], Loss: 0.1217
Epoch [1/20], Step [600/600], Loss: 0.0873
Epoch [2/20], Step [100/600], Loss: 0.1038
Epoch [2/20], Step [200/600], Loss: 0.1665
Epoch [2/20], Step [300/600], Loss: 0.0832
Epoch [2/20], Step [400/600], Loss: 0.0529
Epoch [2/20], Step [500/600], Loss: 0.1128
Epoch [2/20], Step [600/600], Loss: 0.0981
Epoch [3/20], Step [100/600], Loss: 0.1198
Epoch [3/20], Step [200/600], Loss: 0.0254
Epoch [3/20], Step [300/600], Loss: 0.0807
Epoch [3/20], Step [400/600], Loss: 0.0322
Epoch [3/20], Step [500/600], Loss: 0.0597
Epoch [3/20], Step [600/600], Loss: 0.0337
Epoch [4/20], Step [100/600], Loss: 0.0440
Epoch [4/20], Step [200/600], Loss: 0.0082
Epoch [4/20], Step [300/600], Loss: 0.1015
Epoch [4/20], Step [400/600], Loss: 0.0830
Epoch [4/20], Step [500/600], Loss: 0.0601
Epoch [4/20

In [13]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()

  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.180 %
